In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Decomposing BTC into its factors. Using factors: gold, interest rates, bond yields, momentum, inflation, liquidity. Using methods: PCA, Ridge, Lasso, ElasticNet, Best Subset

In [48]:
# gold data 
iau = yf.download("IAU", start="2005-01-01", end="2023-01-01", period="1mo").pct_change().dropna()
iau.head(1)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2005-01-31,-0.011701,-0.009829,-0.011069,-0.009838,-0.009838,-0.737061


In [64]:
# btc data
btc = yf.download("BTC-USD", start="2005-01-01", end="2023-01-01", interval="1mo").pct_change().dropna()
btc.head(1)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-11-01,-0.1259,0.110263,0.108297,0.117421,0.117421,-0.269394


In [42]:
# interest rates
rf = pd.read_csv("Data/InterestRates.csv", index_col="DATE",  parse_dates=True, dtype=float)
rf.head(1)

,INTDSRUSM193N
DATE,
1950-01-01,1.5


In [41]:
# bond yields
by = pd.read_csv("Data/BondYields.csv", index_col="DATE",  parse_dates=True, dtype=float).pct_change().dropna()
by.head(1)

,IRLTLT01USM156N
DATE,
1953-05-01,0.077739


In [43]:
# cpi 
cpi = pd.read_csv("Data/CPI.csv", index_col="DATE",  parse_dates=True, dtype=float)
cpi.head(1)

,CPALTT01USM657N
DATE,
1955-02-01,0.0


In [46]:
# ppi
ppi = pd.read_csv("Data/PPI.csv", index_col="DATE",  parse_dates=True, dtype=float)
ppi.head(1)

,PCUOMFGOMFG
DATE,
1986-01-01,101.3


In [59]:
# liquidity m1
m1 = pd.read_csv("Data/M1.csv", index_col="DATE",  parse_dates=True, dtype=float).pct_change().dropna()
m1 = m1.resample("ME").apply(lambda x: (1+x).prod()-1)
m1.index = m1.index + pd.offsets.MonthBegin(1)
m1.head(1)

,WM1NS
DATE,
1975-02-01,-0.05445
